In [20]:
import numpy as np
import h5py

hf_x = h5py.File('../data/logspec_x.h5', 'r')
x_train = hf_x.get('x_train_data')[()]
x_train = x_train.reshape((64727, 99, 81))
x_train = np.stack((x_train,) * 3, axis=-1)

hf_y = h5py.File('../data/logspec_y.h5', 'r')
y_train = hf_y.get('y_train_data')[()]
#y_train = np.argmax(np.array(y_train), axis=1)

(x_train.shape, y_train.shape)

((64727, 99, 81, 3), (64727, 12))

In [21]:
from tensorflow.keras.layers import GRU, GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, Conv3D, ConvLSTM2D
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16

In [22]:
vgg16_model = VGG16(include_top=False, weights='imagenet', input_shape=(99, 81, 3))

In [23]:
for layer in vgg16_model.layers[:5]:
    layer.trainable = False

x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(12, activation="softmax")(x)

model = Model(vgg16_model.input, predictions)
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 99, 81, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 99, 81, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 99, 81, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 49, 40, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 49, 40, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 49, 40, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 20, 128)       0   

In [ ]:
model.fit(x_train, y_train, batch_size = 32, epochs = 10, validation_split = 0.1)

Train on 58254 samples, validate on 6473 samples
Epoch 1/10
58254/58254 [==============================] - 3722s 64ms/sample - loss: 1.4792 - categorical_accuracy: 0.6622 - val_loss: 5.1229 - val_categorical_accuracy: 0.3671
Epoch 2/10
42848/58254 [=====================>........] - ETA: 16:59 - loss: 1.3739 - categorical_accuracy: 0.6619